- No difference between Linear superposition for 3D structure (with 2 layers at least) and compressing it down to 1 layer and mans phi. Probably need to do one good test with thick and non trivial z-change. 
- need to see how i can recreate some of the crossings that I'm seeing
- start putting things into slides

jesuz christ. so turns out mask can't be just ones... also I made the phase from linsup match the mansuripur algorithm, not that its necessarily right but at least its consistent now. So need to double check and make sure everything works, but then I'll be ready to start doing crossings. lol

Make an S -shape with a bar underneath it, trying to recreate the D biskrymion image

In [1]:
# %matplotlib notebook
%matplotlib qt
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import sys as sys
sys.path.append("../PyTIE/")
from comp_phase import mansPhi, linsupPhi
from TIE_helper import *
from sim_helper import *

In [3]:
np.set_printoptions(linewidth=110) 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

In [4]:
dim = 64
del_px = 16 # nm/pix

b0 = 1e4 #Gauss default 1e4
phi0 = 2.07e7 #Gauss*nm^2 

thickness = 20 #nm
lin_pre_B = 2*np.pi*b0/phi0 * thickness * del_px #thickness is thickness/pixel in nm 

# Test with simple structures

## single layer skyrmion

In [7]:
mag_x, mag_y, mag_z = Bloch(dim, chirality = 'cw', pad = False, ir=0)
mag_x = mag_x.reshape(dim,dim,1)
mag_y = mag_y.reshape(dim,dim,1)
mag_z = mag_z.reshape(dim,dim,1)
q_bloch = winding(mag_x[:,:,0], mag_y[:,:,0], mag_z[:,:,0])

4.173655123685298


In [499]:
show_3D(mag_x, mag_y, mag_z, show_all = False)

In [238]:
show_2D(mag_x[:,:,0], mag_y[:,:,0])

In [255]:
# Mansuripur algorithm
ephi, mphi = std_mansPhi(mag_x[:,:,0], mag_y[:,:,0], 
                         del_px = del_px, b0=b0)

In [253]:
show_im(mphi, title = 'MansPhi')
print(np.max(mphi), np.min(mphi))

16.448415204937866 -5.068932353386093


In [246]:
# Linear superposition
ephil, mphil = linsupPhi(mx=mag_x, my=mag_y, mz=mag_z, 
                       Dshp=np.ones(mag_x.shape), pre_B=lin_pre_B)

nelems =  4096
0.0%
24.4%
48.8%
73.2%
97.7%


In [247]:
show_im(mphil, title = "linsup")
print(np.max(mphil), np.min(mphil))

16.448667209976982 -5.07421129111004


#### Calculate images and reconstruct

In [248]:
defval = 100_000
Tphi, im_ov, im_in, im_un = sim_images(mphil, ephil, del_px = del_px, 
                                       def_val = defval, add_random = False)
show_sims(Tphi, im_ov, im_in, im_un)
print(np.max(im_ov), np.min(im_ov))

Total fov is 1024 nm
0.8851845989781038 0.8394367713177051


In [249]:
recon_tifs = [im_un, im_in, im_ov]
pscope = Microscope(E=200e3, Cs = 200.0e3, theta_c = 0.01e-3, def_spr = 80.0)
ptie = SIM_params(recon_tifs[1],[defval], data_loc = 'blah', flip = False)
# ptie.mask = np.ones(im_ov.shape)
ptie.make_mask(recon_tifs)
ptie.scale = del_px # nm/pix

dataname = f'test'
sym = False
qc = None #0.005
save = False #'b'

results = tie_solver(ptie, recon_tifs, defval, pscope, 
                     dataname = dataname+'_', sym=sym, qc = qc, save=save)

aligning for defocus value:  100000  with no unflipped tfs
Reconstructing with normal Laplacian method
calling tie solver

Reconstructing with sim_params
Phase reconstruction completed.


In [256]:
show_im(results['phase_m'])

## triple layer

In [453]:
mag_x, mag_y, mag_z = Bloch(dim, chirality = 'cw', pad = True, ir=0)
mag_x = mag_x.reshape(dim,dim,1)
mag_y = mag_y.reshape(dim,dim,1)
mag_z = mag_z.reshape(dim,dim,1)
magx3 = np.tile(mag_x, (1,1,3))
magy3 = np.tile(mag_y, (1,1,3))
magz3 = np.tile(mag_z, (1,1,3))
magx3.shape

(64, 64, 3)

In [455]:
show_3D(magx3,magy3,magz3, show_all = True)

In [274]:
ephi_3, mphi_3 = linsupPhi(mx = magx3, my=magy3, mz=magz3, 
                           Dshp = np.ones(magx3.shape), pre_B = lin_pre_B)

nelems =  12288
0.0%
8.14%
16.3%
24.4%
32.6%
40.7%
48.8%
57.0%
65.1%
73.2%
81.4%
89.5%
97.7%


In [277]:
defval = 10_000
Tphi_3, im_ov_3, im_in_3, im_un_3 = sim_images(mphi_3, ephi_3, del_px = del_px, def_val = defval)
show_sims(Tphi_3, im_ov_3, im_in_3, im_un_3)
print(np.max(im_ov_3), np.min(im_ov_3))

Total fov is 1024 nm
0.8839461454885921 0.8629569516073297


#### Calculate images and reconstruct

In [278]:
recon_tifs = [im_un_3, im_in_3, im_ov_3]
pscope = Microscope(E=200e3, Cs = 200.0e3, theta_c = 0.01e-3, def_spr = 80.0)
ptie = SIM_params(recon_tifs[1],[defval], data_loc = 'blah', flip = False)
# ptie.mask = np.ones(im_ov.shape)
ptie.make_mask(recon_tifs)
ptie.scale = del_px # nm/pix

dataname = f'test'
sym = False
qc = None #0.005
save = False #'b'

results = tie_solver(ptie, recon_tifs, defval, pscope, 
                     dataname = dataname+'_', sym=sym, qc = qc, save=save)

aligning for defocus value:  10000  with no unflipped tfs
Reconstructing with normal Laplacian method
calling tie solver

Reconstructing with sim_params
Phase reconstruction completed.


## Test/uniform magnetization comparisons

In [602]:
dim = 64
del_px = 16 # nm/pix

b0 = 1e4 #Gauss default 1e4
phi0 = 2.07e7 #Gauss*nm^2 

thickness = 0.5 #nm per z layer
lin_pre_B = 2*np.pi*b0/phi0 * thickness * del_px #thickness is thickness/pixel in nm 

In [630]:
d = 50
mag_x = np.zeros([dim,dim,d])
mag_y = np.zeros([dim,dim,d])
mag_z = np.zeros([dim,dim,d])

# mag_x[dim//4:3*dim//4,dim//4:3*dim//4,:] = 1
# mag_y[dim//4:3*dim//4,dim//4:3*dim//4,:] = 0

for i in range(d):
    mag_z[dim//4:3*dim//4,dim//4:3*dim//4,i] = 0.25*np.cos(np.pi*i/d)
    mag_x[dim//4:3*dim//4,dim//4:3*dim//4,i] = np.sin(np.pi*i/d)
    mag_y[dim//4:3*dim//4,dim//4:3*dim//4,i] = np.sin(2*np.pi*i/d)


Dshp = np.zeros([dim,dim,d])
Dshp[dim//4:3*dim//4,dim//4:3*dim//4,:] = 1

# mag_z[32:64,32:96] = 1
# mag_z[32:64,64:96] = -1
# mag_z[64:96,32:96] = .5
# mag_z[64:96,64:96] = -.5

In [614]:
show_3D(mag_x*Dshp,mag_y*Dshp,mag_z*Dshp, show_all = True, za = 5)

if compressing to 2D

In [634]:
mag_x = np.sum(mag_x, axis = 2)
mag_y = np.sum(mag_y, axis = 2)
mag_z = np.sum(mag_z, axis = 2)
Dshp = Dshp[:,:,0]

mag_x = mag_x.reshape(dim,dim,1)
mag_y = mag_y.reshape(dim,dim,1)
mag_z = mag_z.reshape(dim,dim,1)
Dshp = Dshp.reshape(dim,dim,1)

In [616]:
show_2D(mag_x[:,:,0], mag_y[:,:,0])

In [640]:
# Mansuripur algorithm
ephi, mphi = std_mansPhi(mag_x[:,:,0], mag_y[:,:,0], 
                         del_px = del_px, b0=b0/(20/thickness))

Linsup calculating

In [635]:
ephi, mphi = linsupPhi(mx = mag_x, my=mag_y, mz=mag_z, Dshp = Dshp, pre_B = lin_pre_B)

nelems =  1024
0.0%
97.7%


In [642]:
path = '../../sim_data/Linsup/through_z/'
defval = 50_000
Tphi, im_ov, im_in, im_un = sim_images(mphi, ephi, del_px = del_px, 
                                       def_val = defval, path = path)
show_sims(Tphi, im_ov, im_in, im_un)
print(np.max(im_ov), np.min(im_ov))

Total fov is 1024 nm
saving to ../../sim_data/Linsup/through_z/
0.9057893622460705 0.8334990743415445


#### Calculate images and reconstruct

In [643]:
recon_tifs = [im_un, im_in, im_ov]
pscope = Microscope(E=200e3, Cs = 200.0e3, theta_c = 0.01e-3, def_spr = 80.0)
ptie = SIM_params(recon_tifs[1],[defval], data_loc = path, flip = False)
# ptie.mask = np.ones(im_ov.shape)
ptie.make_mask(recon_tifs)
ptie.scale = del_px # nm/pix

dataname = f'through_z_50_compressed_Mans'
sym = False
qc = None #0.005
save = 'b'

results = tie_solver(ptie, recon_tifs, defval, pscope, 
                     dataname = dataname+'_', sym=sym, qc = qc, save=save)

aligning for defocus value:  50000  with no unflipped tfs
Reconstructing with normal Laplacian method
calling tie solver

Reconstructing with sim_params
saving images
Phase reconstruction completed.


In [256]:
show_im(results['phase_m'])

# Crossing patterns 

## Crossing bloch skyrmions

In [844]:
dim = 64

In [845]:
rad = dim//2
print(rad)

32


In [859]:
ir = 0

mcw = Bloch(dim, chirality = 'cw', pad = False, ir=ir)
for mag in mcw:
    mag = mag.reshape(dim,dim,1)
# mcw = np.rollaxis(np.rollaxis(np.array(mcw), axis = 2), axis=2)
mcw = np.array(mcw)

mccw = Bloch(dim, chirality = 'ccw', pad = False, ir=ir)
for mag in mccw:
    mag = mag.reshape(dim,dim,1)
# mccw = np.rollaxis(np.rollaxis(np.array(mccw), axis = 2), axis=2)
mccw = np.array(mccw)

In [866]:
d = 1
mag_x = np.zeros([2*dim,2*dim,d])
mag_y = np.zeros([2*dim,2*dim,d])
mag_z = -1*np.zeros([2*dim,2*dim,d])

sep = dim - 15

c2 = [dim,dim//2 + sep,0]
c1 = [dim,3*dim//2 - sep,1]

mag_x[c1[0]-rad:c1[0]+rad,c1[1]-rad:c1[1]+rad,0] += mcw[0]
mag_x[c2[0]-rad:c2[0]+rad,c2[1]-rad:c2[1]+rad,0] += mccw[0]

mag_y[c1[0]-rad:c1[0]+rad,c1[1]-rad:c1[1]+rad,0] += mcw[1]
mag_y[c2[0]-rad:c2[0]+rad,c2[1]-rad:c2[1]+rad,0] += mccw[1]

mag_z[c1[0]-rad:c1[0]+rad,c1[1]-rad:c1[1]+rad,0] += mcw[2]
mag_z[c2[0]-rad:c2[0]+rad,c2[1]-rad:c2[1]+rad,0] += mccw[2]

bnd = 5
Dshp = np.zeros(mag_x.shape)

y1 = c1[0]-rad-bnd
y2 = c2[0]+rad+bnd
x1 = c1[1] - rad - bnd
x2 = c2[1] + rad + bnd
Dshp[int(y1):int(y2), int(x1):int(x2)] = 1

In [871]:
show_3D(mag_x,mag_y,mag_z, a = 20, l = 5,  show_all = True)

In [843]:
show_3D(mag_x*Dshp,mag_y*Dshp,mag_z*Dshp, a = 20,  show_all = True)

if compressing to 2D

In [872]:
mag_x = np.sum(mag_x, axis = 2)
mag_y = np.sum(mag_y, axis = 2)
mag_z = np.sum(mag_z, axis = 2)
Dshp = Dshp[:,:,0]

mag_x = mag_x.reshape(2*dim,2*dim,1)
mag_y = mag_y.reshape(2*dim,2*dim,1)
mag_z = mag_z.reshape(2*dim,2*dim,1)
Dshp = Dshp.reshape(2*dim,2*dim,1)

In [863]:
show_3D(mag_x,mag_y,mag_z, a = 20,  show_all = True)

In [857]:
show_2D(mag_x[:,:,0], mag_y[:,:,0])

In [873]:
q_bisk = winding(mag_x[:,:,0], mag_y[:,:,0], mag_z[:,:,0])

10.316069952677658


In [874]:
# Mansuripur algorithm
ephi, mphi = std_mansPhi(mag_x[:,:,0], mag_y[:,:,0], 
                         del_px = del_px, b0=b0)

#### calculating phase and reconstructing 

In [384]:
ephi, mphi = linsupPhi(mx = mag_x, my=mag_y, mz=mag_z, Dshp = Dshp, pre_B = lin_pre_B)

nelems =  17464
0.0%
5.73%
11.5%
17.2%
22.9%
28.6%
34.4%
40.1%
45.8%
51.5%
57.3%
63.0%
68.7%
74.4%
80.2%
85.9%
91.6%
97.3%


In [875]:
path = '../../sim_data/Linsup/3D_biskyrmion_2/'
defval = 100_000
Tphi, im_ov, im_in, im_un = sim_images(mphi, ephi, del_px = del_px, 
                                       def_val = defval, path = None)
show_sims(Tphi, im_ov, im_in, im_un)
print(np.max(im_ov), np.min(im_ov))

Total fov is 2048 nm
0.9175676210633743 0.8243970156257237


In [876]:
recon_tifs = [im_un, im_in, im_ov]
pscope = Microscope(E=200e3, Cs = 200.0e3, theta_c = 0.01e-3, def_spr = 80.0)
ptie = SIM_params(recon_tifs[1],[defval], data_loc = path, flip = False)
# ptie.mask = np.ones(im_ov.shape)
ptie.make_mask(recon_tifs)
ptie.scale = del_px # nm/pix

dataname = f'2layer_biskyrmion_narrow_compressed2D_Mans'
sym = False
qc = None #0.005
save = False#'b'

results = tie_solver(ptie, recon_tifs, defval, pscope, 
                     dataname = dataname+'_', sym=sym, qc = qc, save=save)

aligning for defocus value:  100000  with no unflipped tfs
Reconstructing with normal Laplacian method
calling tie solver

Reconstructing with sim_params
Phase reconstruction completed.


In [877]:
show_im(results['bbt'])

# Calculate winding number of mag

In [827]:
mag_x, mag_y, mag_z = Bloch(64, chirality = 'ccw', pad = True, ir=0)
# mag_x = mag_x.reshape(dim,dim,1)
# mag_y = mag_y.reshape(dim,dim,1)
# mag_z = mag_z.reshape(dim,dim,1)

In [808]:
mag_x = np.zeros([dim,dim])
mag_y = np.zeros([dim,dim])
mag_z = -1*np.ones([dim,dim])

a=20
b=5
mag_x[a:-a,a:a+b] = -1
mag_x[a:-a,-(b+a):-a] = 1
mag_y[a:a+b,a:-a] = -1
mag_y[-b-a:-a,a:-a] = 1
mag_z[a:-a,a:-a] = 0
mag_z[a+b:-(a+b),a+b:-(a+b)] = 1

#winding number = 1, changes sign with z flipping. 

In [809]:
show_3D(mag_x, mag_y,mag_z)

In [6]:
def winding(mag_x, mag_y, mag_z):
    grad_xy, grad_xx = np.gradient(mag_x)
    grad_yy, grad_yx = np.gradient(mag_y)
    grad_zy, grad_zx = np.gradient(mag_y)

    cp_x = grad_yx*grad_zy - grad_yy * grad_zx
    cp_y = - grad_xx*grad_zy + grad_xy*grad_zx
    cp_z = grad_xx*grad_yy - grad_yx*grad_xy

    # show_im(cp_x, title = "cp_x")
    # show_im(cp_y, title = "cp_y")
    # show_im(cp_z, title = "cp_z")

    dp = mag_x*cp_x + mag_y*cp_y + mag_z*cp_z
    # show_im(dp)
    q = np.sum(dp)
    print(q)
    return q

In [828]:
winding(mag_x, mag_y, mag_z)

4.159922678009682


4.159922678009682

In [667]:
show_im(mag_x)

In [671]:
show_im(grad_xx)

In [672]:
show_im(grad_yy)